# Training & Deploying an Image Classifier using the Built-in Image Classification Algorithm

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

session = sagemaker.Session()
role = get_execution_role()
region_name = boto3.Session().region_name

In [ ]:
%store -r s3_bucket
%store -r prefix

In [ ]:
s3_train_data = 's3://{}/{}/{}'.format(
    s3_bucket, 
    prefix, 
    "train"
)
s3_validation_data = 's3://{}/{}/{}'.format(
    s3_bucket, 
    prefix, 
    "validation"
)
s3_train_lst_path = 's3://{}/{}/{}'.format(
    s3_bucket, 
    prefix, 
    "train_lst"
)
s3_validation_lst_path = 's3://{}/{}/{}'.format(
    s3_bucket, 
    prefix, 
    "validation_lst"
)
s3_output_location = 's3://{}/{}/output'.format(
    s3_bucket, 
    prefix
)

In [ ]:
from sagemaker.image_uris import retrieve 

container = retrieve(
    "image-classification", 
    region_name, 
    "1"
)
container

In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.p2.xlarge',
    output_path=s3_output_location,
    sagemaker_session=session
)

In [ ]:
estimator.set_hyperparameters(
    num_layers=18,
    image_shape = "1,28,28",
    num_classes=10,
    num_training_samples=600,
    mini_batch_size=20,
    epochs=5,
    learning_rate=0.01,
    top_k=2,
    precision_dtype='float32'
)

In [ ]:
from sagemaker.inputs import TrainingInput

train = TrainingInput(
    s3_train_data, 
    distribution='FullyReplicated', 
    content_type='application/x-image', 
    s3_data_type='S3Prefix'
)

validation = TrainingInput(
    s3_validation_data, 
    distribution='FullyReplicated', 
    content_type='application/x-image', 
    s3_data_type='S3Prefix'
)

In [ ]:
content_type = 'application/x-image'

train_lst = TrainingInput(
    s3_train_lst_path, 
    distribution='FullyReplicated', 
    content_type=content_type, 
    s3_data_type='S3Prefix'
)

validation_lst = TrainingInput(
    s3_validation_lst_path, 
    distribution='FullyReplicated', 
    content_type=content_type, 
    s3_data_type='S3Prefix'
)

In [ ]:
data_channels = {
    'train': train, 
    'validation': validation,
    'train_lst': train_lst,
    'validation_lst': validation_lst
}

In [ ]:
%%time

estimator.fit(inputs=data_channels, logs=True)

In [ ]:
endpoint = estimator.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.m4.xlarge'
)

In [ ]:
from sagemaker.serializers import IdentitySerializer

endpoint.serializer = IdentitySerializer(
    content_type="application/x-image"
)

In [ ]:
import json

def get_class_from_results(results):
    results_prob_list = json.loads(results)
    best_index = results_prob_list.index(
        max(results_prob_list)
    )
    
    return {
        0: "ZERO",
        1: "ONE",
        2: "TWO",
        3: "THREE",
        4: "FOUR",
        5: "FIVE",
        6: "SIX",
        7: "SEVEN",
        8: "EIGHT",
        9: "NINE"
    }[best_index]

In [ ]:
from IPython.display import Image, display

def predict(filename, endpoint=endpoint):
    byte_array_input = None
    
    with open(filename, 'rb') as image:
        f = image.read()
        byte_array_input = bytearray(f)
        
    display(Image(filename))
        
    results = endpoint.predict(byte_array_input)
    return get_class_from_results(results)

In [ ]:
!ls tmp/test

In [ ]:
results = !ls -1 tmp/test
for filename in results:
    print(predict(f"tmp/test/{filename}"))

In [ ]:
endpoint.delete_endpoint()

In [ ]:
tn = estimator.latest_training_job.name
training_job_name = tn
%store training_job_name